In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse

In [2]:
data = pd.read_csv('Gangnam.csv', encoding='cp949')

In [3]:
# 음식과 관광호텔을 제외한 행만 필터링
filtered_data = data[(data['KTO카테고리대분류명'] != '음식') & (data['KTO카테고리소분류명'] != '관광호텔')]

In [4]:
# 관광지명 추출
tourist_spots = filtered_data['관광지명'].tolist()

# 관광지명을 URL 인코딩하여 리스트에 저장
encoded_list = [urllib.parse.quote(s) for s in tourist_spots]

In [5]:
# 새로운 데이터프레임 생성
new_data = pd.DataFrame(columns=['시군구명', '관광지명', '음식점명', '음식점카테고리'])

# 중복된 음식점을 추적하기 위한 세트 생성
unique_restaurants = set()

# 음식점 정보 가져오기
for spot, row in zip(encoded_list, filtered_data.iterrows()):
    basic = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={spot}+%EB%A7%9B%EC%A7%91'
    url = requests.get(basic, headers={'User-Agent': 'Mozilla/5.0'})
    html = url.content
    soup = BeautifulSoup(html, 'html.parser')

    # 음식점명 + 카테고리 추출
    span_name = soup.find_all(class_='TYaxT')
    span_category = soup.find_all(class_='KCMnt')

    # 각 음식점명과 카테고리를 묶어서 음식점 정보 생성
    for name, category in zip(span_name, span_category):
        restaurant_name = name.text.strip()
        restaurant_category = category.text.strip()
        
        # 중복된 음식점인지 확인하고 중복되지 않은 경우에만 데이터프레임에 추가
        if (row[1]['관광지명'], restaurant_name) not in unique_restaurants:
            new_data = pd.concat([new_data, pd.DataFrame({'시군구명': [row[1]['시군구명']],
                                                          '관광지명': [row[1]['관광지명']],
                                                          '음식점명': [restaurant_name],
                                                          '음식점카테고리': [restaurant_category]})], ignore_index=True)
            # 중복 제거 세트에 추가
            unique_restaurants.add((row[1]['관광지명'], restaurant_name))

# 새로운 데이터프레임 출력
print(new_data)

   시군구명       관광지명       음식점명  음식점카테고리
0   강남구  스타필드 코엑스몰         미랑      양갈비
1   강남구  스타필드 코엑스몰         지칸     이자카야
2   강남구  스타필드 코엑스몰         루아     이자카야
3   강남구  스타필드 코엑스몰   김이정 숯불구이  육류,고기요리
4   강남구  스타필드 코엑스몰  동어동락 삼성본점      생선회
..  ...        ...        ...      ...
91  강남구       학동공원  트라가 가로수길점    스페인음식
92  강남구       학동공원     정돈 신사점      돈가스
93  강남구       학동공원    미미면가 본점      일식당
94  강남구       학동공원    카이센동우니도      일식당
95  강남구       학동공원    클랩피자 청담       피자

[96 rows x 4 columns]
